# Lesson 3: Show the data as it is acquired

In this lesson, we'll show how to use the tools provided with Bluesky to show the data as it is acquired using both a table representation and a graphical view, as well.  These capabilities are provided by using *callbacks*.  In lessons 1 and 2, we wrote our own simple callback to view the documents that come from the RunEngine during execution of a plan.  Quickly, the data became too complex for simple viewing.

The *LiveTable* and *LivePlot* callbacks provide a table and graphical view of the data from the plan.  We'll get to those first.  Later, we'll show the *BestEffortCallback*, which combines both those callbacks plus a little more.  For routine work, we'll want to use *BestEffortCallback* all the time.  We'll show how to make that happen so we *set it and forget about it*.

----------------------

Starting with the configuration from lessons 1 and 2, we first group the *imports* together as is common Python practice:

In [1]:
from ophyd import EpicsMotor
from ophyd.scaler import ScalerCH
from bluesky import RunEngine
import bluesky.plans as bp
from APS_BlueSky_tools.devices import use_EPICS_scaler_channels

Next, make a RunEngine (for scanning) and connect our motor and scaler

In [2]:
RE = RunEngine({})
m1 = EpicsMotor("prj:m1", name="m1")
scaler = ScalerCH("prj:scaler1", name="scaler")

Reconfigure the scaler for channel names and show the result.

In [4]:
scaler.channels.chan01.chname.put("clock")
scaler.channels.chan02.chname.put("I0")
scaler.channels.chan03.chname.put("scint")

scaler.match_names()
use_EPICS_scaler_channels(scaler)
scaler.read()

OrderedDict([('clock', {'value': 11000000.0, 'timestamp': 1533083197.141285}),
             ('I0', {'value': 5.0, 'timestamp': 1533083197.141285}),
             ('scint', {'value': 2.0, 'timestamp': 1533083197.141285}),
             ('scaler_time', {'value': 1.1, 'timestamp': 1533083197.141285})])

## Showing the data

In [5]:
from bluesky.callbacks import LiveTable

In [6]:
RE(bp.scan([scaler], m1, 1, 5, 5), LiveTable([scaler]))

RuntimeError: This event loop is already running

+-----------+------------+------------+------------+-------------+------------+
|   seq_num |       time |         I0 |      clock | scaler_time |      scint |
+-----------+------------+------------+------------+-------------+------------+
|         1 | 19:45:12.8 |          5 |   11000000 |       1.100 |          3 |
|         2 | 19:45:15.2 |          6 |   11000000 |       1.100 |          5 |
|         3 | 19:45:17.6 |          5 |   11000000 |       1.100 |          5 |
|         4 | 19:45:20.0 |          4 |   11000000 |       1.100 |          7 |
|         5 | 19:45:22.4 |          2 |   11000000 |       1.100 |          5 |
+-----------+------------+------------+------------+-------------+------------+
generator scan ['f38d5023'] (scan num: 1)


RE.subscribe(LiveTable([scaler]))

In [ ]:
RE(bp.scan([scaler], m1, 1, 5, 5))

---------
see http://nsls-ii.github.io/bluesky/callbacks.html#aside-making-plots-update-live

In [ ]:
%matplotlib notebook
from bluesky.utils import install_nb_kicker
install_nb_kicker()

In [ ]:
from bluesky.callbacks import LivePlot

In [ ]:
RE(bp.scan([scaler], m1, 1, 5, 5), LivePlot("scaler", "m1"))

In [ ]:
RE.subscribe(LivePlot("scaler", "m1"))

In [ ]:
RE(bp.scan([scaler], m1, 1, 5, 5))

In [ ]:
RE.unsubscribe_all()

------------------
see http://nsls-ii.github.io/bluesky/callbacks.html#best-effort-callback

In [ ]:
from bluesky.callbacks.best_effort import BestEffortCallback

In [ ]:
RE(bp.scan([scaler], m1, 1, 5, 5), BestEffortCallback())

In [ ]:
RE.subscribe(BestEffortCallback())

In [ ]:
RE(bp.scan([scaler], m1, 1, 5, 5))